In [1]:
import scipy.sparse as sps
from Builder import Builder
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample, split_train_in_two_percentage_global_sample_k_fold
from Recommenders.XGboostRecommender import XGboostRecommender
import pickle
from Evaluation.Evaluator import EvaluatorHoldout
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from tqdm import tqdm
from Recommenders.GeneralizedLinearHybridRecommender import GeneralizedLinearHybridRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask as MultVAERecommender
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
import pickle

from Recommenders.ScoresHybridRecommender import ScoresHybridRecommender


saved_urm_path = 'saved_models'
saved_models_path = f'{saved_urm_path}/train_val'
builder = Builder()
builder.preprocess()

URM = sps.load_npz(f'{saved_urm_path}/URM.npz')
URM_train = sps.load_npz(f'{saved_urm_path}/URM_train.npz')
URM_val = sps.load_npz(f'{saved_urm_path}/URM_val.npz')
URM_train_val = URM_train + URM_val
URM_test = sps.load_npz(f'{saved_urm_path}/URM_test.npz')
eval = EvaluatorHoldout(URM_val, cutoff_list=[10, 35, 40])
etest = EvaluatorHoldout(URM_test, cutoff_list=[10, 35, 40])

with open('best_models_info.pickle', 'rb') as f:
    best_models_info = pickle.load(f)
    
def evaluate(rec, validator=etest, metric='MAP', cutoff=10):
    result_df, _ = validator.evaluateRecommender(rec)
    return result_df[metric][cutoff]
    

/Users/lorecampa/anaconda3/envs/RecSysFramework/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


EvaluatorHoldout: Ignoring 2650 (21.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 2136 (16.9%) Users that have less than 1 test interactions


In [2]:
from Recommenders.NonPersonalizedRecommender import TopPop


top_pop = TopPop(URM_train_val)
top_pop.fit()
evaluate(top_pop)

TopPopRecommender: URM Detected 225 ( 1.8%) users with no interactions.
TopPopRecommender: URM Detected 116 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10502 (100.0%) in 18.78 sec. Users per second: 559


0.011742921983114276

In [3]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender

item_knn= ItemKNNCFRecommender(URM_train_val)
item_knn.fit(**best_models_info['ItemKNNCFRecommenderCrossValNDCG']['hs'])
evaluate(item_knn)

ItemKNNCFRecommender: URM Detected 225 ( 1.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 116 ( 0.5%) items with no interactions.
Similarity column 22222 (100.0%), 2823.83 column/sec. Elapsed time 7.87 sec
EvaluatorHoldout: Processed 10502 (100.0%) in 16.26 sec. Users per second: 646


0.03773176613373752

In [4]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender

user_knn= UserKNNCFRecommender(URM_train_val)
user_knn.fit(**best_models_info['UserKNNCFRecommenderCrossVal']['hs'])
evaluate(user_knn)

UserKNNCFRecommender: URM Detected 225 ( 1.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 116 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 2902.24 column/sec. Elapsed time 4.35 sec
EvaluatorHoldout: Processed 10502 (100.0%) in 19.07 sec. Users per second: 551


0.03497897981639208

In [5]:
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

p3alpha= P3alphaRecommender(URM_train_val)
p3alpha.fit(**best_models_info['P3alphaRecommenderCrossVal']['hs'])
evaluate(p3alpha)

P3alphaRecommender: URM Detected 225 ( 1.8%) users with no interactions.
P3alphaRecommender: URM Detected 116 ( 0.5%) items with no interactions.
P3alphaRecommender: Similarity column 22222 (100.0%), 5974.33 column/sec. Elapsed time 3.72 sec
EvaluatorHoldout: Processed 10502 (100.0%) in 5.53 sec. Users per second: 1898


0.04794156744142517

In [6]:
rp3_beta = RP3betaRecommender(URM_train_val)
rp3_beta.fit(**best_models_info['RP3betaRecommenderCrossVal']['hs'])
evaluate(rp3_beta)

RP3betaRecommender: URM Detected 225 ( 1.8%) users with no interactions.
RP3betaRecommender: URM Detected 116 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 5800.94 column/sec. Elapsed time 3.83 sec
EvaluatorHoldout: Processed 10502 (100.0%) in 5.71 sec. Users per second: 1839


0.04899704289130172

In [7]:
ials = IALSRecommender(URM_train_val)
ials.load_model(f'{saved_models_path}')
evaluate(ials)


IALSRecommender: URM Detected 225 ( 1.8%) users with no interactions.
IALSRecommender: URM Detected 116 ( 0.5%) items with no interactions.
IALSRecommender: Loading model from file 'saved_models/train_valIALSRecommender'
IALSRecommender: Loading complete
EvaluatorHoldout: Processed 10502 (100.0%) in 10.22 sec. Users per second: 1028


0.03230756122038771

In [10]:
ials.ITEM_factors.shape

(22222, 80)

In [8]:
multvae = MultVAERecommender(URM_train_val)
multvae.load_model(f'{saved_models_path}')
evaluate(multvae)

MultVAERecommender: URM Detected 225 ( 1.8%) users with no interactions.
MultVAERecommender: URM Detected 116 ( 0.5%) items with no interactions.
MultVAERecommender: Loading model from file 'saved_models/train_val/MultVAERecommender'
INFO:tensorflow:Restoring parameters from saved_models/train_val/MultVAERecommender/.session/session


2024-02-21 10:32:21.038833: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled


MultVAERecommender: Loading complete
EvaluatorHoldout: Processed 10502 (100.0%) in 6.13 sec. Users per second: 1712


0.03786835765220872

In [9]:
slim = SLIMElasticNetRecommender(URM_train_val)
slim.fit(**best_models_info['SLIMElasticNetRecommenderCrossVal']['hs'])
evaluate(slim)

SLIMElasticNetRecommender: URM Detected 225 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 116 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 21004 (94.5%) in 5.00 min. Items per second: 70.01
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 5.38 min. Items per second: 68.86
EvaluatorHoldout: Processed 10502 (100.0%) in 4.80 sec. Users per second: 2190


0.05003893438286965

In [10]:
slim_bpr = SLIM_BPR_Cython(URM_train_val)
slim_bpr.fit(**best_models_info['SLIM_BPR']['hs'])
evaluate(slim_bpr)

SLIM_BPR_Recommender: URM Detected 225 ( 1.8%) users with no interactions.
SLIM_BPR_Recommender: URM Detected 116 ( 0.5%) items with no interactions.
Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.


/bin/sh: free: command not found


Processed 12638 (100.0%) in 0.94 sec. BPR loss is 4.42E-02. Sample per second: 13440
SLIM_BPR_Recommender: Epoch 1 of 110. Elapsed time 0.13 sec
Processed 12638 (100.0%) in 1.14 sec. BPR loss is 1.26E-01. Sample per second: 11066
SLIM_BPR_Recommender: Epoch 2 of 110. Elapsed time 0.33 sec
Processed 12638 (100.0%) in 0.34 sec. BPR loss is 2.13E-01. Sample per second: 37120
SLIM_BPR_Recommender: Epoch 3 of 110. Elapsed time 0.53 sec
Processed 12638 (100.0%) in 0.54 sec. BPR loss is 2.38E-01. Sample per second: 23528
SLIM_BPR_Recommender: Epoch 4 of 110. Elapsed time 0.73 sec
Processed 12638 (100.0%) in 0.72 sec. BPR loss is 3.03E-01. Sample per second: 17582
SLIM_BPR_Recommender: Epoch 5 of 110. Elapsed time 0.91 sec
Processed 12638 (100.0%) in 0.91 sec. BPR loss is 2.96E-01. Sample per second: 13811
SLIM_BPR_Recommender: Epoch 6 of 110. Elapsed time 1.11 sec
Processed 12638 (100.0%) in 1.14 sec. BPR loss is 4.46E-01. Sample per second: 11021
SLIM_BPR_Recommender: Epoch 7 of 110. Elapsed

0.038933872761953675

In [12]:
hybrid_scores_slim_rp3_map = ScoresHybridRecommender(URM_train_val, slim, rp3_beta)
hybrid_scores_slim_rp3_map.fit(**best_models_info['ScoresHybridRecommenderCrossVal']['hs'])
evaluate(hybrid_scores_slim_rp3_map)

ScoresHybridRecommender: URM Detected 225 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 116 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10502 (100.0%) in 5.63 sec. Users per second: 1865


0.051042900369695804

In [25]:
slim_recall = SLIMElasticNetRecommender(URM_train_val)
slim_recall.load_model(f'{saved_models_path}', file_name='SLIMElasticNetRecommenderCrossValRecall40')
rp3_beta_recall = SLIMElasticNetRecommender(URM_train_val)
rp3_beta_recall.load_model(f'{saved_models_path}', file_name='RP3betaRecommenderCrossValRecall40')
hybrid_scores_slim_rp3_recall = ScoresHybridRecommender(URM_train_val, slim_recall, rp3_beta_recall)
hybrid_scores_slim_rp3_recall.fit(**best_models_info['ScoresHybridRecommenderCrossValRecall40']['hs'])
evaluate(hybrid_scores_slim_rp3_recall, metric='RECALL', cutoff=35), evaluate(slim_recall, metric='RECALL', cutoff=35), evaluate(rp3_beta_recall, metric='RECALL', cutoff=35)

SLIMElasticNetRecommender: URM Detected 225 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 116 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'saved_models/train_valSLIMElasticNetRecommenderCrossValRecall40'
SLIMElasticNetRecommender: Loading complete
SLIMElasticNetRecommender: URM Detected 225 ( 1.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 116 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Loading model from file 'saved_models/train_valRP3betaRecommenderCrossValRecall40'
SLIMElasticNetRecommender: Loading complete
ScoresHybridRecommender: URM Detected 225 ( 1.8%) users with no interactions.
ScoresHybridRecommender: URM Detected 116 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 10502 (100.0%) in 9.01 sec. Users per second: 1165
EvaluatorHoldout: Processed 10502 (100.0%) in 9.09 sec. Users per second: 1155
EvaluatorHoldout: Processed 10502 (100.0%) in 5.52 

(0.24885661208159643, 0.2422121339136158, 0.24156948706145084)